<a href="https://colab.research.google.com/github/LordofD-MBrown/2021_ERAU_REU_NNSS_Variable_Stride_Algorithm_Analysis/blob/main/ERAU_REU_2021_NNSS_PROJECT_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ERAU REU 2021 NNSS PROJECT

Programmed By: Matthew Brown

Programmed In: Python using Google Colab


In [ ]:
#Connecting Program to Google Drive
#=====================================================
from google.colab import drive
drive.mount('/gdrive')
#=====================================================

Mounted at /gdrive


In [ ]:
#Imports
#===============================================================================
import cv2
import keras
import matplotlib
import pickle
import os
import random
import sklearn
import time
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from keras import activations
from keras import backend as K
from PIL import Image
from sklearn import metrics
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Flatten, Layer
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#===============================================================================

The following code is gathering the data and turning it into an array that the neural network can read. The image sizes have been preset to be 600 x 600. There are 200 entries

In [ ]:
#Compiling Data to Test With
#===============================================================================
Data_Location = '/gdrive/My Drive/ERAU_REU_SUMMER_2021/ERAU_retinopathy_data_rotated'
Classifications = ["Class 1", "Class 4"]

training_data = []

def initialize_training_data():
  for classification in Classifications:
    path = os.path.join(Data_Location, classification)
    classification_num = Classifications.index(classification) # 0 = Class 1, 1 = Class 4
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path, img))
      training_data.append([img_array, classification_num])
initialize_training_data()
random.shuffle(training_data) #Makes sure there is no order to the data set

Image_Array = []
Label_Array = []

for features, label in training_data:
  Image_Array.append(features)
  Label_Array.append(label)

Image_Array = np.array(Image_Array).reshape(-1,600,600,3) #Note the last value is 3 as it is in RGB
#===============================================================================

In [ ]:
pickle_out = open("Image_Array_0.pickle", "wb")
pickle.dump(Image_Array, pickle_out)
pickle_out.close()

pickle_out = open("Label_Array_0.pickle", "wb")
pickle.dump(Label_Array, pickle_out)
pickle_out.close()

# Max Pool Pre-Processing

In [ ]:
#Max Pool Pre-Processing
#===============================================================================
pickle_in = open("Image_Array_0.pickle", "rb")
Image_Array = pickle.load(pickle_in)
pickle_in = open("Label_Array_0.pickle", "rb")
Label_Array = pickle.load(pickle_in)
#Scaling the Variables
#===================================
Image_Array = np.array(Image_Array / 255.0)
Label_Array = np.array(Label_Array)
#===================================

#Indexing
#==========================================
image_number = 0
#==========================================

#Setting Parameters
#==========================================
filter_x = 2
filter_y = 2
filter_z = 3                                                                    #filter size 2x2x3
input_shape = (Image_Array.shape)
nx = input_shape[1]
ny = input_shape[2]
nz = input_shape[3]

output_dim = (int(input_shape[1] / filter_x), int(input_shape[2] / filter_y), filter_z)
print(output_dim)
post_vs_image_array = np.zeros((Image_Array.shape[0], ) + output_dim)                 
#==========================================
while image_number < Image_Array.shape[0]:
  print(image_number)  
  input_image = Image_Array[image_number]                                       #Successful
  post_vs_image = np.zeros(output_dim)                                          #Successful sets to (332,240,3)      

  oldrow = 0
  oldcol = 0

  new_col_idx = 0
  new_row_idx = 0

  while (oldrow < nx - 1 and new_row_idx < post_vs_image.shape[0]):

    while (oldcol < ny - 1 and new_col_idx < post_vs_image.shape[1]):

      temp_r = tf.keras.backend.get_value(input_image[oldrow:oldrow + filter_x,oldcol:oldcol + filter_y,0])
      temp_g = tf.keras.backend.get_value(input_image[oldrow:oldrow + filter_x,oldcol:oldcol + filter_y,1])
      temp_b = tf.keras.backend.get_value(input_image[oldrow:oldrow + filter_x,oldcol:oldcol + filter_y,2])

      max_r = np.max(temp_r)
      max_g = np.max(temp_g)
      max_b = np.max(temp_b)

      post_vs_image[new_row_idx, new_col_idx, 0] = max_r
      post_vs_image[new_row_idx, new_col_idx, 1] = max_g
      post_vs_image[new_row_idx, new_col_idx, 2] = max_b

      #=========================================================================
      oldcol += filter_y
      new_col_idx += 1
    oldcol = 0
    new_col_idx = 0
    oldrow += filter_x
    new_row_idx += 1
    #===========================================================================
  post_vs_image_array[image_number] = post_vs_image
  image_number += 1

In [ ]:
#Saving Pickles
#===============================================================================
#tf.keras.preprocessing.image.array_to_img(Image_Array[0], scale = True)
tf.keras.preprocessing.image.array_to_img(post_vs_image_array[1], scale = True)
print()
pickle_out = open("max_pool_image_array.pickle", "wb")
pickle.dump(post_vs_image_array, pickle_out)
pickle_out.close()

pickle_out = open("label_array_1.pickle", "wb")
pickle.dump(Label_Array, pickle_out)
pickle_out.close()
#===============================================================================

# Avg Pool Pre-Processing

In [ ]:
#Avg Pool Pre-Processing
#===============================================================================
pickle_in = open("Image_Array_0.pickle", "rb")
Image_Array = pickle.load(pickle_in)
pickle_in = open("Label_Array_0.pickle", "rb")
Label_Array = pickle.load(pickle_in)
#Scaling the Variables
#===================================
Image_Array = np.array(Image_Array / 255.0)
Label_Array = np.array(Label_Array)
#===================================

#Indexing
#==========================================
image_number = 0
#==========================================

#Setting Parameters
#==========================================
filter_x = 2
filter_y = 2
filter_z = 3                                                                    #filter size 2x2x3
input_shape = (Image_Array.shape)
nx = input_shape[1]
ny = input_shape[2]
nz = input_shape[3] 

output_dim = (int(input_shape[1] / filter_x), int(input_shape[2] / filter_y), filter_z)
print(output_dim)
post_vs_image_array = np.zeros((Image_Array.shape[0], ) + output_dim)                 
#==========================================
while image_number < Image_Array.shape[0]:
  print(image_number)  
  input_image = Image_Array[image_number]                                       #Successful
  post_vs_image = np.zeros(output_dim)                                          #Successful sets to (332,240,3)      

  oldrow = 0
  oldcol = 0

  new_col_idx = 0
  new_row_idx = 0

  while (oldrow < nx - 1 and new_row_idx < post_vs_image.shape[0]):

    while (oldcol < ny - 1 and new_col_idx < post_vs_image.shape[1]):

      temp_r = tf.keras.backend.get_value(input_image[oldrow:oldrow + filter_x,oldcol:oldcol + filter_y,0])
      temp_g = tf.keras.backend.get_value(input_image[oldrow:oldrow + filter_x,oldcol:oldcol + filter_y,1])
      temp_b = tf.keras.backend.get_value(input_image[oldrow:oldrow + filter_x,oldcol:oldcol + filter_y,2])

      avg_r = np.average(temp_r)
      avg_g = np.average(temp_g)
      avg_b = np.average(temp_b)

      post_vs_image[new_row_idx, new_col_idx, 0] = avg_r
      post_vs_image[new_row_idx, new_col_idx, 1] = avg_g
      post_vs_image[new_row_idx, new_col_idx, 2] = avg_b

      #=========================================================================
      oldcol += filter_y
      new_col_idx += 1
    oldcol = 0
    new_col_idx = 0
    oldrow += filter_x
    new_row_idx += 1
    #===========================================================================
  post_vs_image_array[image_number] = post_vs_image
  image_number += 1

In [ ]:
#tf.keras.preprocessing.image.array_to_img(Image_Array[0], scale = True)
tf.keras.preprocessing.image.array_to_img(post_vs_image_array[1], scale = True)
print(Label_Array[1])
#pickle_out = open("average_pool_image.pickle", "wb")
#pickle.dump(post_vs_image_array, pickle_out)
#pickle_out.close()

0


# Variable Stide Pre-Processing#
This code a pre-processing version of variable stride that can be implemented to the images before they are entered in the CNN.

In [ ]:
#Caluculating Variable Stride Out Shape
#===============================================================================
def CalcVSOutShape(input_shape,              #(q,x,y,z)
                   step_v = [4, 1, 4],       #Vertical Step
                   step_h = [4, 1, 4],          #Horizontal Step
                   frac_v = [1/3, 1/3, 1/3], 
                   frac_h = [1/3, 1/3, 1/3]):
  #print('Input Shape:', input_shape)
  (q, nx, ny, nz) = input_shape                 #Sets the nx, ny, nz = input x,y,z
  #print(nx)                                 #600
  #print(ny)                                 #600
  #print(nz)                                 #3
  cum_frac_v = np.cumsum(frac_v)             #Return the cumulative sum of the elements along a given axis.
  cum_frac_h = np.cumsum(frac_h)
  #print(cum_fraction_v)                     #[0.33333333 0.66666667 1.]
  #print(cum_fraction_h)                     #[0.25 1.]
  frac_bounds_v = cum_frac_v * (nx-0) - 1
  frac_bounds_h = cum_frac_h * (ny-0) - 1
  #print(frac_bounds_v)                      #[199. 399. 599.]
  #print(frac_bounds_h)                      #[149. 599.]

  out_h = 0
  out_v = 0
  idx_low = 0

  for frac_idx in range(len(frac_h)):
    idx_span = min(frac_bounds_h[frac_idx], ny - 3) - idx_low                   
    divisor_h, remander_h = divmod(idx_span, step_h[frac_idx])
    out_h += int(divisor_h) + 1
    idx_low += (int(divisor_h) + 1) * step_h[frac_idx]

  idx_low = 0

  for frac_idx in range(len(frac_v)):
    idx_span = min(frac_bounds_v[frac_idx], nx - 3) - idx_low  
    divisor_v, remainder_v = divmod(idx_span, step_v[frac_idx])
    out_v += int(divisor_v) + 1
    idx_low += (int(divisor_v) + 1) * step_v[frac_idx]

  #print(out_v)             #332
  #print(out_h)             #240 
  #print(nz)                #3
  return (out_v, out_h, nz)
#===============================================================================

#Variable Stride
#===============================================================================
pickle_in = open("Image_Array_0.pickle", "rb")
Image_Array = pickle.load(pickle_in)
pickle_in = open("Label_Array_0.pickle", "rb")
Label_Array = pickle.load(pickle_in)
#Scaling the Variables
#===================================
Image_Array = np.array(Image_Array / 255.0)
Label_Array = np.array(Label_Array)
#===================================

#Indexing
#==========================================
image_number = 0
#==========================================

#Setting Parameters
#==========================================
k1 = np.array([[1, 2, 1],
               [2, 4, 2],
               [1, 2, 1]])
step_v = [2, 3, 1, 3, 2]      #Vertical Step
step_h = [3,2]       #Horizontal Step
frac_v = [1/5, 1/5, 1/5, 1/5, 1/5]
frac_h = [1/5, 4/5]

input_shape = (Image_Array.shape)
nx = input_shape[1]
ny = input_shape[2]
nz = input_shape[3] 

output_dim = CalcVSOutShape(input_shape, step_h = step_h, step_v = step_v, frac_v = frac_v, frac_h = frac_h)
print(output_dim)
kernel = np.repeat(k1[:,:,np.newaxis], nz, axis = 2)
post_vs_image_array = np.zeros((Image_Array.shape[0], ) + output_dim)                 
#==========================================
while image_number < Image_Array.shape[0]:
  print(image_number)
  input_image = Image_Array[image_number]                                       #Successful
  post_vs_image = np.zeros(output_dim)                                          #Successful sets to (332,240,3)      

  mystep_v = 0
  mystep_h = 0
  myfrac_v = 0
  myfrac_h = 0

  oldrow = 0
  oldcol = 0

  new_col_idx = 0
  new_row_idx = 0

  while (oldrow < nx - 2 and new_row_idx < post_vs_image.shape[0]):  
    while (oldcol < ny - 2 and new_col_idx < post_vs_image.shape[1]):                    
      temp = tf.keras.backend.get_value(input_image[oldrow:oldrow + 3, oldcol:oldcol + 3, :])
      temp = (temp * kernel)                                                           
      post_vs_image[new_row_idx, new_col_idx, :] = np.sum(np.sum(temp,axis = 0), axis = 0)
      #=========================================================================
      if oldcol > (np.sum(frac_h[0:myfrac_h + 1]) * nx - 1):
        mystep_h += 1
        myfrac_h += 1
      oldcol += step_h[mystep_h]
      new_col_idx += 1
    if oldrow > (np.sum(frac_v[0:myfrac_v + 1]) * ny - 1):
      mystep_v += 1
      myfrac_v += 1
    oldcol = 0
    mystep_h = 0
    myfrac_h = 0
    oldrow += step_v[mystep_v]
    new_col_idx = 0
    new_row_idx += 1
    #===========================================================================
  post_vs_image_array[image_number] = post_vs_image
  image_number += 1

In [ ]:
#tf.keras.preprocessing.image.array_to_img(Image_Array[0], scale = True)
pickle_in = open("variable_stride_custom_image_array.pickle", "rb")
x = pickle.load(pickle_in)
tf.keras.preprocessing.image.array_to_img(x[0], scale = True)
#pickle_out = open("variable_stride_custom_image_array.pickle", "wb")
#pickle.dump(post_vs_image_array, pickle_out)
#pickle_out.close()

Saving the Image Array:

After running the code on a new stride method, you must save the following pickle file to you computer then reupload it next session to keep data.

In [ ]:
pickle_out = open("Image_Array.pickle", "wb")
pickle.dump(Image_Array, pickle_out)
pickle_out.close()

pickle_out = open("label_array.pickle", "wb")
pickle.dump(Label_Array, pickle_out)
pickle_out.close()

# Reset the TensorBoard

In [ ]:
#Refresh Logs
#===============================================================================
%rm -rf logs #Reset Logs
#===============================================================================

# **The Convolutional Neural Network**

In [ ]:
#The Convolutional Neural Network
#===============================================================================
pickle_in = open("Image_Array_0.pickle", "rb")
x = pickle.load(pickle_in)
pickle_in = open("Label_Array_0.pickle", "rb")
y = pickle.load(pickle_in)
#============================logs of progress===================================
logdir = os.path.join("logs", "7_64_64_64_32_16_1_Dropout_.7") #Edit Name of Logs
tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
#===============================================================================

model = Sequential()

#Layer - Convolutional Layer
#===================================
model.add(Conv2D(64, (5,5), activation = 'relu', input_shape = (300, 300, 3)))  #Check
model.add(MaxPooling2D(pool_size=(2,2)))
#===================================

#Layer - Convolutional Layer
#===================================
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(Dropout(.2))
model.add(MaxPooling2D(pool_size=(2,2)))
#===================================

#Layer - Convolutional Layer
#===================================
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(Dropout(.2))
model.add(MaxPooling2D(pool_size=(2,2)))
#===================================

#Layer - Convolutional Layer
#===================================
model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(Dropout(.2))
model.add(MaxPooling2D(pool_size=(2,2)))
#===================================

model.add(Flatten())

#Layer - Dense
#===================================
model.add(Dense(32, activation = 'relu'))
#===================================

#Layer - Dense
#===================================
model.add(Dense(16, activation = 'relu'))
#===================================

#Layer - Output
#===================================
model.add(Dense(1))
model.add(Activation('sigmoid'))
#===================================

model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ['accuracy'])
model.fit(x, y, batch_size = 10, epochs = 100, validation_split=.15, callbacks=[tensorboard_callback], shuffle = True)

model.summary()

#Load Tensor Board
#===============================================================================
#%reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir logs
#===============================================================================

# Testing Different Stuctures of Neural Networks

In [ ]:
pickle_in = open("average_pool_image.pickle", "rb")
x = pickle.load(pickle_in)
pickle_in = open("Label_Array_0.pickle", "rb")
y = pickle.load(pickle_in)
tf.keras.preprocessing.image.array_to_img(x[1], scale = True)

In [ ]:
Classifications = ["Class 1", "Class 4"]

dense_layers = [4]                                                              #Adjust dense layers                                                   
layer_sizes = [64]                                                              #Adjust number of Nodes
conv_layers = [5]                                                               #Adjust conv layers
drop_out_rates = [.05]                                                           #Adjust drop out rate
valid_seperation = .2

total_runs = 8
current_run = 0
dim = (40, 319, 279, 3) #266,332                                                  #Adjust input dimensions Avg. Max = 40,300,300,3  Custom = 40,319,279,3  Center = 40,300,300,3 Right = 40,332,266,3

pickle_in = open("variable_stride_custom_image_array.pickle", "rb")
x = pickle.load(pickle_in)
pickle_in = open("Label_Array_0.pickle", "rb")
y = pickle.load(pickle_in)

x = np.array(x)
y = np.array(y)

test_data_1 = int(200 - (200 * valid_seperation))
test_data_2 = int(200 - (200 * valid_seperation))

i = 0

test_data_image = np.zeros(dim)
test_data_label = np.zeros(40)

for test_data_1 in range(test_data_2, 200):
  test_data_image[i] = x[test_data_1]
  test_data_label[i] = y[test_data_1]
  i += 1

for current_run in range(0, total_runs):
  for dense_layer in dense_layers:
    for layer_size in layer_sizes:
      for conv_layer in conv_layers:
        for drop_out in drop_out_rates:
          NAME = "{}-conv-{}-nodes-{}-dense-{}-dropout-{}-run-{}".format(conv_layer, layer_size, dense_layer,drop_out, (current_run + 1), int(time.time()))

          print(NAME)
          model = Sequential()

          model.add(Conv2D(layer_size, (3,3), input_shape = (319, 279, 3)))      #Adjust input dimensions Avg. Max = 300,300,3  Custom = 319,279,3  Center = 300,300,3 Right = 332,266,3
          model.add(Activation("relu"))
          model.add(MaxPooling2D(pool_size=(2,2)))
          model.add(Dropout(drop_out))

          for l in range(conv_layer - 1):
            model.add(Conv2D(layer_size, (3, 3)))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(drop_out))
      
          model.add(Flatten())

          for l in range(dense_layer):
            model.add(Dense(layer_size))
            model.add(Activation('relu'))

          model.add(Dense(1))
          model.add(Activation('sigmoid'))

          #============================logs of progress===================================
          logdir = os.path.join("logs", NAME) #Edit Name of Logs
          tensorboard_callback = TensorBoard(logdir, histogram_freq=1)
          stop = tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', patience = 10)
          #===============================================================================

          model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ['accuracy'])
          model.fit(x, y, batch_size = 10, epochs = 150, validation_split = valid_seperation, callbacks = [tensorboard_callback], shuffle = True)

          model.summary()

          label_pred_keras = model.predict(test_data_image, batch_size = 1)
          fpr_keras, tpr_keras, thresholds_keras = sklearn.metrics.roc_curve(test_data_label, label_pred_keras)
          auc = sklearn.metrics.roc_auc_score(test_data_label, label_pred_keras)

          plt.figure(1)
          plt.plot([0, 1], [0, 1], 'k--')
          plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc))
          plt.xlabel('False positive rate')
          plt.ylabel('True positive rate')
          plt.title('ROC curve')
          plt.legend(loc='best')
          plt.show()

          for n in range(np.max(np.size(label_pred_keras))):
            if label_pred_keras[n,0] < .5:
              label_pred_keras[n,0] = 0
            if label_pred_keras[n,0] >= 0.5:
              label_pred_keras[n,0] = 1
            
          print('Classification Report')

          print(sklearn.metrics.classification_report(test_data_label,label_pred_keras,target_names = Classifications))

#Load Tensor Board
#===============================================================================
#%reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir logs
#===============================================================================

In [ ]:
#Load Tensor Board
#===============================================================================
#%reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir logs
#===============================================================================